In [ ]:
import pandas as pd
import numpy as np

In [ ]:
datas = pd.read_csv('P3_GrantExport.csv',sep=';')

In [ ]:
data_final = datas[[0,7,11,12,13]]
data_final.head()

In [ ]:
data_final=data_final.rename(columns={'\ufeff"Project Number\"':'Project Number'})
data_final.head()

In [ ]:
data_final.set_index('Project Number')
data_final.head()

Some data need to be replaced with NaN, as they do not convey any information regarding the location of the grant that was assigned. These are the following :
- 'Nicht zuteilbar - NA' -> Unavailable data
- 'NPO (Biblioth., Museen, Verwalt.) - NPO' -> No informations
- 'Weitere Institute - FINST' -> Other institutes
- 'Firmen/Privatwirtschaft - FP' -> Private institutions

In [ ]:
data_final = data_final.replace('Nicht zuteilbar - NA',np.NaN)
data_final = data_final.replace('NPO (Biblioth., Museen, Verwalt.) - NPO',np.NaN)
data_final = data_final.replace('Weitere Institute - FINST',np.NaN)
data_final = data_final.replace('Firmen/Privatwirtschaft - FP',np.NaN)

data_uni = data_final['University'].unique()



data_uni

In [ ]:
test = data_uni[47].split('-')
print(test)

In [ ]:
#A INSTALLER AVEC pip install geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode(test[0])
print(location.address)
print((location.latitude, location.longitude))
print(location.raw)